# Food Delivery Data Analysis
### Innomatics Research Labs – Advanced GenAI Internship

In [3]:
import pandas as pd
import json
import sqlite3

In [5]:
orders = pd.read_csv("orders.csv")

with open("users.json", "r") as f:
    users = pd.DataFrame(json.load(f))

conn = sqlite3.connect(":memory:")
with open("restaurants.sql", "r") as f:
    conn.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

In [7]:
final_df = orders.merge(users, on="user_id", how="left") \
                 .merge(restaurants, on="restaurant_id", how="left")

final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [9]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [19]:
import pandas as pd

# Load final dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

print(df.columns)

# Q1: City with highest total revenue from Gold members
q1 = df[df["membership"] == "Gold"].groupby("city")["total_amount"].sum()
print("Q1 - Gold revenue by city:\n", q1, "\n")
print("ANSWER:", q1.idxmax(), "\n")

# Q2: Cuisine with highest average order value
q2 = df.groupby("cuisine")["total_amount"].mean()
print("Q2 - Average order value by cuisine:\n", q2, "\n")
print("ANSWER:", q2.idxmax(), "\n")

# Q3: Distinct users with total order value > 1000
user_totals = df.groupby("user_id")["total_amount"].sum()
count_users = user_totals[user_totals > 1000].count()
print("Q3 - Users with total orders > 1000:", count_users, "\n")

# Q4: Rating range with highest revenue
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]
df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels, include_lowest=True)
q4 = df.groupby("rating_range")["total_amount"].sum()
print("Q4 - Revenue by rating range:\n", q4, "\n")
print("ANSWER:", q4.idxmax(), "\n")

# Q5: City with highest AOV among Gold members
q5 = df[df["membership"] == "Gold"].groupby("city")["total_amount"].mean()
print("Q5 - Gold AOV by city:\n", q5, "\n")
print("ANSWER:", q5.idxmax(), "\n")

# Q6: Cuisine with lowest restaurant count but high revenue
restaurant_count = df.groupby("cuisine")["restaurant_id"].nunique()
revenue = df.groupby("cuisine")["total_amount"].sum()
ratio = revenue / restaurant_count
print("Q6 - Revenue per restaurant:\n", ratio, "\n")
print("ANSWER:", ratio.idxmax(), "\n")

# Q7: Percentage of orders by Gold members
gold_percentage = round((df[df["membership"] == "Gold"].shape[0] / df.shape[0]) * 100)
print("Q7 - Percentage of orders by Gold members:", gold_percentage, "%\n")

# Q8: Restaurant with highest average order value but < 20 total orders

restaurant_stats = df.groupby("restaurant_name_y").agg(
    avg_order=("total_amount", "mean"),
    total_orders=("order_id", "count")
)

q8 = restaurant_stats[restaurant_stats["total_orders"] < 20] \
        .sort_values("avg_order", ascending=False)

print("Q8 - Restaurants with <20 orders:\n", q8.head(), "\n")
print("ANSWER:", q8.index[0], "\n")
df[df["restaurant_name_y"] == "Restaurant_294"][["restaurant_name_x"]].head()

# Q9: Membership + Cuisine with highest revenue
q9 = df.groupby(["membership", "cuisine"])["total_amount"].sum()
print("Q9 - Revenue by membership + cuisine:\n", q9, "\n")
print("ANSWER:", q9.idxmax(), "\n")

# Q10: Quarter with highest revenue
df["order_date"] = pd.to_datetime(df["order_date"])
df["quarter"] = df["order_date"].dt.quarter
q10 = df.groupby("quarter")["total_amount"].sum()
print("Q10 - Revenue by quarter:\n", q10, "\n")
print("ANSWER: Q", q10.idxmax(), sep="")


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')
Q1 - Gold revenue by city:
 city
Bangalore     994702.59
Chennai      1080909.79
Hyderabad     896740.19
Pune         1003012.32
Name: total_amount, dtype: float64 

ANSWER: Chennai 

Q2 - Average order value by cuisine:
 cuisine
Chinese    798.389020
Indian     798.466011
Italian    799.448578
Mexican    808.021344
Name: total_amount, dtype: float64 

ANSWER: Mexican 

Q3 - Users with total orders > 1000: 2544 

Q4 - Revenue by rating range:
 rating_range
3.0–3.5    2136772.70
3.6–4.0    1717494.41
4.1–4.5    1960326.26
4.6–5.0    2197030.75
Name: total_amount, dtype: float64 

ANSWER: 4.6–5.0 

Q5 - Gold AOV by city:
 city
Bangalore    793.223756
Chennai      808.459080
Hyderabad    806.421034
Pune         781.162243
Name: total_amount, dtype: float64 

ANSWER: Chennai 

Q6 - Reve

C:\Users\koppu\AppData\Local\Temp\ipykernel_7364\401273137.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  q4 = df.groupby("rating_range")["total_amount"].sum()
C:\Users\koppu\AppData\Local\Temp\ipykernel_7364\401273137.py:67: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


In [13]:
#️ How many total orders were placed by users with Gold membership?
gold_orders = df[df["membership"] == "Gold"].shape[0]
print("1. Total orders by Gold members:", gold_orders)

# 2️ Total revenue generated from Hyderabad (rounded)
hyderabad_revenue = round(df[df["city"] == "Hyderabad"]["total_amount"].sum())
print("2. Total revenue from Hyderabad:", hyderabad_revenue)

# 3️ How many distinct users placed at least one order?
distinct_users = df["user_id"].nunique()
print("3. Distinct users who placed at least one order:", distinct_users)

# 4️ Average order value for Gold members (rounded to 2 decimals)
gold_aov = round(df[df["membership"] == "Gold"]["total_amount"].mean(), 2)
print("4. Average order value for Gold members:", gold_aov)

# 5️ Orders placed for restaurants with rating ≥ 4.5
high_rating_orders = df[df["rating"] >= 4.5].shape[0]
print("5. Orders with restaurant rating ≥ 4.5:", high_rating_orders)

# 6️ Orders placed in the top revenue city among Gold members
top_gold_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

orders_in_top_city = df[
    (df["membership"] == "Gold") & (df["city"] == top_gold_city)
].shape[0]

print("6. Top revenue city among Gold members:", top_gold_city)
print("   Orders placed in that city by Gold members:", orders_in_top_city)

Object `membership` not found.
1. Total orders by Gold members: 4987
2. Total revenue from Hyderabad: 1889367
3. Distinct users who placed at least one order: 2883
4. Average order value for Gold members: 797.15
5. Orders with restaurant rating ≥ 4.5: 3374
6. Top revenue city among Gold members: Chennai
   Orders placed in that city by Gold members: 1337
